In [11]:
import openai
import pandas as pd

OPEN_API_KEY = ''

openai.api_key = OPEN_API_KEY

def get_completion(paragraph, level):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "[역할] 당신은 초등학교 국어 선생님입니다. 해당 지문을 확인한 후 4지선다 문제를 출제하세요."},
                {"role": "user", "content": f"[방법]"
                                            f" 아래 주어진 지문 내용을 바탕으로 {level} 수준의 4지선다 형태의 객관식 문제 1개를 출제하세요. 반드시 아래 Output의 유형에 맞는 구성으로 만드세요. 요구한 결과물만 출력해줘"
                                            f"[Input] "
                                            f"[지문]  "
                                            f"{paragraph} "
                                            f"[난이도] - {level}"
                                            f""
                                            f"[Output]"
                                            f"{example}"}
            ],
            model="gpt-4o-mini"
        )
        # print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

news = pd.read_csv("../data/processed/news/result/news_level.csv", sep=',')

example = """
{
    "question": "주어진 지문의 주제로 알맞는 것은?",
    "options":  {
        "1":"보기A",
        "2":"보기b",
        "3":"보기c",
        "4":"보기d"
        },
    "answer": 2,
    "reason": "답에 대한 이유"
}
"""

responses = []
level_mapping = {
    1: "미취학아동",
    2: "초등학교 1학년",
    3: "초등학교 2학년",
    4: "초등학교 3학년",
    5: "초등학교 4학년",
    6: "초등학교 5학년",
    7: "초등학교 6학년"
}

for i, row in news.iterrows():
    if (i+1)%50==0:
        print(i+1)
    education_level = level_mapping.get(row['Difficulty_Level'], "Unknown")
    response = get_completion(row['preprocessed_content'], education_level)
    
    if response:
        # print(response)
        responses.append(response)
    else:
        responses.append("Failed to get a response from the API.")
        
# print(responses)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850


In [12]:
responses_df = pd.DataFrame(news[['URL','preprocessed_content', 'Difficulty_Level']])
responses_df['response'] = responses

responses_df.to_csv("../data/processed/news/dt02_21.csv", index=False, encoding='utf-8-sig')

In [5]:
# import pandas as pd
# responses_df = pd.read_csv("../data/processed/news/result/news_level.csv", sep=',')
# responses_df['textID'] = ['n{:04d}'.format(i+1) for i in range(len(responses_df))]
# responses_df.to_csv("../data/processed/news/result/news_level.csv", index=False, encoding='utf-8-sig')

In [1]:
#불러오기
import pandas as pd
import json
from datetime import datetime

responses_df = pd.read_csv("../data/processed/news/dt02_21.csv", sep=',',encoding='cp949')
news = pd.read_csv("../data/processed/news/result/news_level.csv", sep=',')
content_to_textID = dict(zip(news['preprocessed_content'], news['textID']))

print(responses_df['response'].head())
transformed_data = []

def get_textID(content):
    return content_to_textID.get(content, None)

def transform_row(row):
    response_json = json.loads(row['response'])
    options = response_json.get('options', {})
    
    answer_key = str(response_json.get("answer", ""))  
    answer_value = ""
    for k, v in options.items():
        if str(k) == answer_key:
            answer_value = v  
            break
    
    now = datetime.now().isoformat()

    transformed = {
        "title": response_json.get("question", ""),
        "content": row['preprocessed_content'],
        "answer" : answer_value,
        "solution": response_json.get("reason", ""),
        "questionTypeCode": "QT01",
        "problemLevelCode": f"PL0{row['Difficulty_Level']}",
        "problemTypeCode": "PT02",
        "problemTypeDetailCode": "PT0221",
        "metadata": {
            "options": [v for v in options.values()] 
        },
        "createdDate": now,
        "modifiedDate": now
    }
    
    return transformed


for i, row in responses_df.iterrows():
    try:
        json.loads(row['response'])  # Try parsing the original JSON
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
        print(f"Problematic JSON: {row['response']}")

dt02_21_df = pd.DataFrame(transformed_data)

0    {\n    "question": "지문에서 등장하는 풍선은 무엇인가요?",\n  ...
1    {\n    "question": "추석 때 주의해야 할 점은 무엇인가요?",\n ...
2    {\n    "question": "주어진 지문의 주제로 알맞는 것은?",\n   ...
3    {\n    "question": "2024 파리 패럴림픽에서 종합 1위를 차지한 ...
4    {\n    "question": "에펠탑에 걸린 오륜 조형물에 대한 논란의 주요 ...
Name: response, dtype: object
Error in row 122: Expecting property name enclosed in double quotes: line 8 column 5 (char 169)
Problematic JSON: {
    "question": "주어진 지문의 주제로 알맞는 것은?",
    "options":  {
        "1":"한지의 역사와 문화",
        "2":"한지의 제조 과정과 응용",
        "3":"한지의 색상 변화",
        "4":"한지의 가격과 판매",
    },
    "answer": 2,
    "reason": "지문에서는 한지의 만들기 과정과 이를 활용한 다양한 응용에 대해 설명하고 있기 때문에 '한지의 제조 과정과 응용'이 주제에 가장 알맞습니다."
}
Error in row 148: Expecting property name enclosed in double quotes: line 8 column 5 (char 148)
Problematic JSON: {
    "question": "반려동물이 혼자 있을 때 느끼는 것은 무엇인가요?",
    "options": {
        "1": "행복함",
        "2": "외로움",
        "3": "힘듦",
        "4": "짜증",
    },
    "answer": 2,
    "reason":

In [2]:
print(dt02_21_df)
with open("../data/processed/problem_data/pt0221.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=4, ensure_ascii=False)
dt02_21_df.to_csv("../data/processed/problem_data/pt0221.csv", index=False, encoding='utf-8-sig')

                                    title  \
0                    지문에서 등장하는 풍선은 무엇인가요?   
1                   추석 때 주의해야 할 점은 무엇인가요?   
2                     주어진 지문의 주제로 알맞는 것은?   
3    2024 파리 패럴림픽에서 종합 1위를 차지한 나라는 어디인가요?   
4    에펠탑에 걸린 오륜 조형물에 대한 논란의 주요 내용은 무엇인가요?   
..                                    ...   
869               이 지문에서 이야기하는 기술은 무엇인가요?   
870                   주어진 지문의 주제로 알맞는 것은?   
871    가습기 살균제 사건과 관련하여 법원에서 어떤 결정을 내렸나요?   
872            버츄얼 아이돌의 인기가 높은 이유는 무엇인가요?   
873            케냐에서 주로 잡으려는 바다 생물은 무엇인가요?   

                                               content  \
0    푸른 호수 위에 노란 왕관을 쓴 거대한 랍스터가 둥실둥실 떠있어요. 커다란 집게발이...   
1    추석이 코앞으로 다가왔어요. 추석 때는 맛있는 전과 같은 요리를 준비하며 즐거운 연...   
2    활활 타오르는 태양이 내뿜는 빛은 지구로 쏟아져요. 이런 태양 빛을 모아 전기를 만...   
3    지난달 29일부터 열린 2024 파리 패럴림픽이 12일간의 여정을 마감하고, 9일 ...   
4    오륜은 파란색·검은색·빨간색·노란색·초록색의 크기가 같은 다섯 색깔의 고리를 말해요...   
..                                                 ...   
869  머지않아 '배우'라는 직업이 인공지능(AI)으로 대체될지도 모르겠어요. AI로 영상..

In [22]:
import pandas as pd
import json

news = pd.read_csv("../data/processed/news/result/news_level.csv", sep=',', encoding='cp949')

dt0221 = pd.read_csv("../data/processed/problem_data/pt0221.csv", sep=',')
dt0222 = pd.read_csv("../data/processed/problem_data/pt0222.csv", sep=',')

content_to_url = dict(zip(news['preprocessed_content'], news['URL']))
content_to_classification = dict(zip(news['preprocessed_content'], news['classification']))
content_to_word_count = dict(zip(news['preprocessed_content'], news['word_count']))
content_to_word_avg = dict(zip(news['preprocessed_content'], news['word_avg']))
content_to_word_hard = dict(zip(news['preprocessed_content'], news['word_hard']))
content_to_length = dict(zip(news['preprocessed_content'], news['length']))
content_to_Difficulty_Level = dict(zip(news['preprocessed_content'], news['Difficulty_Level']))

def get_contentID(content):
    return content_to_url.get(content)
def get_classification(content):
    return content_to_classification.get(content)
def get_word_count(content):
    return content_to_word_count.get(content)
def get_word_avg(content):
    return content_to_word_avg.get(content)
def get_word_hard(content):
    return content_to_word_hard.get(content)
def get_length(content):
    return content_to_length.get(content)
def get_Difficulty_Level(content):
    return content_to_Difficulty_Level.get(content)

dt0221['contentID'] = dt0221['content'].apply(get_contentID)
dt0221['classification'] = dt0221['content'].apply(get_classification)
dt0221['word_count'] = dt0221['content'].apply(get_word_count)
dt0221['word_avg'] = dt0221['content'].apply(get_word_avg)
dt0221['word_hard'] = dt0221['content'].apply(get_word_hard)
dt0221['length'] = dt0221['content'].apply(get_length)
dt0221['Difficulty_Level'] = dt0221['content'].apply(get_Difficulty_Level)

dt0222['contentID'] = dt0222['content'].apply(get_contentID)
dt0222['classification'] = dt0222['content'].apply(get_classification)
dt0222['word_count'] = dt0222['content'].apply(get_word_count)
dt0222['word_avg'] = dt0222['content'].apply(get_word_avg)
dt0222['word_hard'] = dt0222['content'].apply(get_word_hard)
dt0222['length'] = dt0222['content'].apply(get_length)
dt0222['Difficulty_Level'] = dt0222['content'].apply(get_Difficulty_Level)

print(dt0221)

                                    title  \
0                    지문에서 등장하는 풍선은 무엇인가요?   
1                   추석 때 주의해야 할 점은 무엇인가요?   
2                     주어진 지문의 주제로 알맞는 것은?   
3    2024 파리 패럴림픽에서 종합 1위를 차지한 나라는 어디인가요?   
4    에펠탑에 걸린 오륜 조형물에 대한 논란의 주요 내용은 무엇인가요?   
..                                    ...   
869               이 지문에서 이야기하는 기술은 무엇인가요?   
870                   주어진 지문의 주제로 알맞는 것은?   
871    가습기 살균제 사건과 관련하여 법원에서 어떤 결정을 내렸나요?   
872            버츄얼 아이돌의 인기가 높은 이유는 무엇인가요?   
873            케냐에서 주로 잡으려는 바다 생물은 무엇인가요?   

                                               content  \
0    푸른 호수 위에 노란 왕관을 쓴 거대한 랍스터가 둥실둥실 떠있어요. 커다란 집게발이...   
1    추석이 코앞으로 다가왔어요. 추석 때는 맛있는 전과 같은 요리를 준비하며 즐거운 연...   
2    활활 타오르는 태양이 내뿜는 빛은 지구로 쏟아져요. 이런 태양 빛을 모아 전기를 만...   
3    지난달 29일부터 열린 2024 파리 패럴림픽이 12일간의 여정을 마감하고, 9일 ...   
4    오륜은 파란색·검은색·빨간색·노란색·초록색의 크기가 같은 다섯 색깔의 고리를 말해요...   
..                                                 ...   
869  머지않아 '배우'라는 직업이 인공지능(AI)으로 대체될지도 모르겠어요. AI로 영상..

In [23]:
from datetime import datetime
def transform_row(row):
    now = datetime.now().isoformat()

    transformed = {
        "title": row['title'],
        "content": row['content'],
        "answer" : row['answer'],
        "solution": row['solution'],
        "questionTypeCode": row['questionTypeCode'],
        "problemLevelCode": row['problemLevelCode'],
        "problemTypeCode": row['problemTypeCode'],
        "problemTypeDetailCode": row['problemTypeDetailCode'],
        "metadata": row['metadata'],
        "createdDate": now,
        "modifiedDate": now,
        "contentID": row['contentID'],
        "classification": row['classification'],
        "wordCount" : row['word_count'],
        "wordAvg" : row['word_avg'],
        "wordHard" : row['word_hard'],
        "length" : row['length'],
        "Difficulty_Level" : row['Difficulty_Level']
    }
    
    return transformed

In [24]:
transformed_data = []
for i, row in dt0221.iterrows():
    try:
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
dt0221_df = pd.DataFrame(transformed_data)

In [25]:
dt0221.to_csv("../data/processed/problem_data/pt0221.csv", index=False, encoding='utf-8-sig')
dt0222.to_csv("../data/processed/problem_data/pt0222.csv", index=False, encoding='utf-8-sig')